__Name : Tahir Fareed__

__Roll no : SU92-BSAIM-F24-036__

__Program: BSAI (3-A)__

 ## __Badminton tournament Ranking system__

In [ ]:
from collections import deque
import heapq

# Linked List Node and List

class Node:
    def __init__(self, player):
        self.player = player
        self.next = None

class PlayerLinkedList:
    def __init__(self):
        self.head = None

    def add(self, player):
        node = Node(player)
        if not self.head:
            self.head = node
        else:
            cur = self.head
            while cur.next:
                cur = cur.next
            cur.next = node

    def remove_by_id(self, player_id):
        prev = None
        cur = self.head
        while cur:
            if cur.player.id == player_id:
                if prev:
                    prev.next = cur.next
                else:
                    self.head = cur.next
                return True
            prev = cur
            cur = cur.next
        return False

    def to_list(self):
        arr = []
        cur = self.head
        while cur:
            arr.append(cur.player)
            cur = cur.next
        return arr

# Player Class

class Player:
    _id = 1
    def __init__(self, name, rating=1000):
        self.id = Player._id
        Player._id += 1
        self.name = name
        self.rating = rating
        self.wins = 0
        self.losses = 0

# Tournament Class
class Tournament:
    def __init__(self):
        self.roster = PlayerLinkedList()  # Linked List
        self.players = {}                 # Hash Map
        self.schedule = deque()           # Queue
        self.undo_stack = []              # Stack
        self.h2h = {}                     # Graph

    # Add player
    def add_player(self, name, rating=1000):
        p = Player(name, rating)
        self.roster.add(p)
        self.players[p.id] = p
        self.h2h[p.id] = {}
        return p.id

    # Remove player
    def remove_player(self, pid):
        if pid in self.players:
            self.roster.remove_by_id(pid)
            del self.players[pid]
            if pid in self.h2h:
                del self.h2h[pid]
            for k in self.h2h:
                if pid in self.h2h[k]:
                    del self.h2h[k][pid]
            return True
        return False

    # Schedule match
    def schedule_match(self, p1, p2):
        self.schedule.append((p1, p2))

    # Record result
    def record_result(self, winner, loser):
        w = self.players[winner]
        l = self.players[loser]

        # Save state for undo (stack)
        self.undo_stack.append((winner, loser, w.wins, l.losses, w.rating, l.rating))

        w.wins += 1
        l.losses += 1
        w.rating += 10
        l.rating = max(0, l.rating - 10)

        # Graph update
        self.h2h[winner][loser] = self.h2h[winner].get(loser, 0) + 1

    # Undo
    def undo(self):
        if not self.undo_stack:
            print("Nothing to undo.")
            return
        winner, loser, ww, ll, wr, lr = self.undo_stack.pop()
        w = self.players[winner]
        l = self.players[loser]
        w.wins = ww
        l.losses = ll
        w.rating = wr
        l.rating = lr
        if self.h2h[winner][loser] > 0:
            self.h2h[winner][loser] -= 1

    # Show rankings (sorted)
    def show_rankings(self):
        ranking = sorted(self.roster.to_list(), key=lambda p: (-p.wins, -p.rating))
        for i, p in enumerate(ranking, 1):
            print(f"{i}. {p.name} (ID {p.id}) - Wins {p.wins} | Rating {p.rating}")

    # Top-K using heap
    def top_k(self, k):
        heap = []
        for p in self.roster.to_list():
            heapq.heappush(heap, (-p.wins, -p.rating, p.id, p))
        print(f"Top {k}:")
        for i in range(min(k, len(heap))):
            _, _, _, p = heapq.heappop(heap)
            print(f"{i+1}. {p.name} - Wins {p.wins}, Rating {p.rating}")


# Main Loop

t = Tournament()

while True:
    print("""
1. Add Player
2. Remove Player
3. Schedule Match
4. Record Result
5. Undo Last Result
6. Show Rankings
7. Show Top-K
0. Exit
""")
    choice = input("Choice: ")

    if choice == "1":
        name = input("Name: ")
        rating = int(input("Rating: "))
        pid = t.add_player(name, rating)
        print("Player Added with ID:", pid)
    elif choice == "2":
        pid = int(input("Player ID to remove: "))
        if t.remove_player(pid):
            print("Player removed.")
        else:
            print("Not found.")
    elif choice == "3":
        p1 = int(input("Player 1 ID: "))
        p2 = int(input("Player 2 ID: "))
        t.schedule_match(p1, p2)
        print("Match Scheduled.")
    elif choice == "4":
        if not t.schedule:
            print("No matches!")
        else:
            p1, p2 = t.schedule.popleft()
            print(f"Match: {t.players[p1].name} vs {t.players[p2].name}")
            winner = int(input("Winner ID: "))
            loser = p1 if winner == p2 else p2
            t.record_result(winner, loser)
            print("Result Recorded.")
    elif choice == "5":
        t.undo()
    elif choice == "6":
        t.show_rankings()
    elif choice == "7":
        k = int(input("Enter K: "))
        t.top_k(k)
    elif choice == "0":
        break
    else:
        print("Invalid choice.")



1. Add Player
2. Remove Player
3. Schedule Match
4. Record Result
5. Undo Last Result
6. Show Rankings
7. Show Top-K
0. Exit



Top 2:

1. Add Player
2. Remove Player
3. Schedule Match
4. Record Result
5. Undo Last Result
6. Show Rankings
7. Show Top-K
0. Exit

